# Deploying My Small Model

In [2]:
#imports
import torch
import torchvision
from torchvision import transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
def load_model(model_path, device):
    """
    Loads a model from the specified file path.

    Args:
    model_path (str): Path to the model file.
    device (str): The device to load the model on ('cuda' or 'cpu').

    Returns:
    torch.nn.Module: Loaded PyTorch model.
    """
    weights = torchvision.models.DenseNet201_Weights.DEFAULT # best available weight
    # Recreate the classifier layer and seed it to the target device
    model = torchvision.models.densenet201(weights=weights).to(device)
    model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1920,
                    out_features=4, # same number of output units as our number of classes
                    bias=True)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def preprocess_image(image_path):
    """
    Preprocesses the image for model prediction.

    Args:
    image_path (str): Path to the image file.

    Returns:
    torch.Tensor: Preprocessed image tensor.
    """
    image = torchvision.io.read_image(image_path).type(torch.float32)
    image = image / 255.0  # Normalize to [0, 1]
    transform = transforms.Resize(size=(64, 64))
    return transform(image)

def get_prediction(model, image, device):
    """
    Predicts the class for the given image using the specified model.

    Args:
    model (torch.nn.Module): The trained model for prediction.
    image (torch.Tensor): The preprocessed image tensor.
    device (str): The device to perform prediction on ('cuda' or 'cpu').

    Returns:
    str, float: Predicted class name and the probability.
    """
    class_names = ['buffalo', 'elephant', 'rhino', 'zebra']
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.inference_mode():
        pred_logits = model(image)
        pred_prob = torch.softmax(pred_logits, dim=1)
        pred_label = torch.argmax(pred_prob, dim=1)
    return class_names[pred_label], pred_prob.max().item()

def predict_image(image_path, model_path):
    """
    Main function to handle model prediction on the given image.

    Args:
    image_path (str): Path to the target image.
    model_path (str): Path to the trained model file.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[INFO] Predicting on image: {image_path}")

    # Load the model and preprocess the image
    model = load_model(model_path, device)
    image = preprocess_image(image_path)

    # Predict and print the result
    pred_class, pred_prob = get_prediction(model, image, device)
    print(f"[INFO] Predicted class: {pred_class}, Probability: {pred_prob:.3f}")

In [5]:
image_path = 'image/buffalo.jpeg'
model_path = 'model/densenetafri.pth'

In [6]:
predict_image(image_path,model_path)

[INFO] Predicting on image: image/buffalo.jpeg


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /home/lukmanaj/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:08<00:00, 9.45MB/s]
/home/lukmanaj/miniconda3/envs/arewads/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[INFO] Predicted class: buffalo, Probability: 0.963


It works. I'm done. We move.